In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.animation as animation
from pathlib import Path
import sys
import os
from datetime import datetime
from scipy.optimize import curve_fit
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks, welch

In [11]:
# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['lines.linewidth'] = 2

In [12]:
class AnalizadorQHO:
    """Clase para analizar datos de simulaciones QHO"""
    
    def __init__(self, archivo_csv):
        """
        Inicializa el analizador con un archivo CSV
        
        Args:
            archivo_csv (str): Ruta al archivo CSV
        """
        self.archivo = archivo_csv
        self.df = pd.read_csv(archivo_csv)
        self.nombre = Path(archivo_csv).stem
        self.tipo = self._detectar_tipo()
        
        print(f"\n{'='*70}")
        print(f"Analizando: {self.nombre}")
        print(f"Tipo: {self.tipo}")
        print(f"Puntos de datos: {len(self.df)}")
        print(f"Tiempo total: {self.df['tiempo'].max():.2f} s")
        print(f"{'='*70}\n")
        
    def _detectar_tipo(self):
        """Detecta el tipo de simulación basándose en las columnas"""
        columnas = set(self.df.columns)
        
        if 'squeezing_r' in columnas:
            return 'estado_comprimido'
        elif 'prob_n0' in columnas:
            return 'superposicion'
        elif 'aceleracion' in columnas:
            return 'oscilador_clasico'
        elif 'alpha_real' in columnas:
            return 'estado_coherente'
        else:
            return 'desconocido'
    
    def analisis_completo(self, guardar=True):
        """
        Realiza un análisis completo de la simulación
        
        Args:
            guardar (bool): Si True, guarda las figuras generadas
        """
        print("📊 Iniciando análisis completo...\n")
        
        # 1. Resumen estadístico
        self.resumen_estadistico()
        
        # 2. Verificaciones físicas
        self.verificar_principios_fisicos()
        
        # 3. Gráficas principales según el tipo
        if self.tipo == 'estado_coherente':
            self.analizar_coherente()
        elif self.tipo == 'estado_comprimido':
            self.analizar_comprimido()
        elif self.tipo == 'superposicion':
            self.analizar_superposicion()
        elif self.tipo == 'oscilador_clasico':
            self.analizar_clasico()
        
        # 4. Guardar figuras si se solicita
        if guardar:
            self.guardar_figuras()
        
        print("\n✅ Análisis completo finalizado!")
    
    def resumen_estadistico(self):
        """Imprime resumen estadístico de los datos"""
        print("📈 RESUMEN ESTADÍSTICO")
        print("-" * 70)
        
        if self.tipo != 'oscilador_clasico':
            # Datos cuánticos
            print(f"\n🔵 Valores esperados:")
            print(f"  ⟨X⟩: {self.df['X_avg'].mean():.4f} ± {self.df['X_avg'].std():.4f}")
            print(f"  ⟨P⟩: {self.df['P_avg'].mean():.4f} ± {self.df['P_avg'].std():.4f}")
            
            print(f"\n🔵 Incertezas:")
            print(f"  ΔX: {self.df['delta_X'].mean():.4f} ± {self.df['delta_X'].std():.4f}")
            print(f"  ΔP: {self.df['delta_P'].mean():.4f} ± {self.df['delta_P'].std():.4f}")
            print(f"  ΔX·ΔP: {self.df['producto_incerteza'].mean():.4f} ± {self.df['producto_incerteza'].std():.4f}")
            
            print(f"\n🔵 Número de fotones:")
            print(f"  ⟨n⟩: {self.df['n_promedio'].mean():.4f} ± {self.df['n_promedio'].std():.4f}")
            print(f"  Mandel Q: {self.df['mandel_Q'].mean():.4f} ± {self.df['mandel_Q'].std():.4f}")
            
            print(f"\n🔵 Información cuántica:")
            print(f"  Pureza: {self.df['pureza'].mean():.6f} ± {self.df['pureza'].std():.6f}")
            print(f"  Entropía: {self.df['entropia'].mean():.6f} ± {self.df['entropia'].std():.6f}")
            
            if self.tipo == 'estado_comprimido':
                print(f"\n🔵 Squeezing:")
                print(f"  r: {self.df['squeezing_r'].iloc[0]:.4f}")
                print(f"  Squeezing (dB): {self.df['squeezing_dB'].mean():.2f} ± {self.df['squeezing_dB'].std():.2f} dB")
        
        else:
            # Datos clásicos
            print(f"\n🔴 Oscilador Clásico:")
            print(f"  Amplitud: {self.df['amplitud'].iloc[0]:.4f}")
            print(f"  Energía: {self.df['energia_total'].mean():.4f} ± {self.df['energia_total'].std():.4f}")
            print(f"  Periodo: {self.df['periodo'].iloc[0]:.4f}")
        
        print("\n" + "-" * 70)
    
    def verificar_principios_fisicos(self):
        """Verifica que se cumplan principios físicos fundamentales"""
        print("\n🔬 VERIFICACIÓN DE PRINCIPIOS FÍSICOS")
        print("-" * 70)
        
        if self.tipo != 'oscilador_clasico':
            # 1. Principio de incertidumbre
            min_producto = self.df['producto_incerteza'].min()
            print(f"\n✓ Principio de incertidumbre:")
            print(f"  ΔX·ΔP mínimo = {min_producto:.6f}")
            if min_producto >= 0.49:  # Tolerancia numérica
                print(f"  ✅ CUMPLE (≥ 0.5)")
            else:
                print(f"  ❌ NO CUMPLE (debería ser ≥ 0.5)")
            
            # 2. Conservación de área (estados puros)
            area_media = self.df['area_elipse'].mean()
            area_std = self.df['area_elipse'].std()
            print(f"\n✓ Conservación de área (Liouville):")
            print(f"  Área = {area_media:.6f} ± {area_std:.6f}")
            if abs(area_media - np.pi) < 0.1:
                print(f"  ✅ CUMPLE (≈ π = {np.pi:.6f})")
            else:
                print(f"  ⚠️  Desviación de π: {abs(area_media - np.pi):.6f}")
            
            # 3. Pureza de estado puro
            pureza_media = self.df['pureza'].mean()
            print(f"\n✓ Estado puro:")
            print(f"  Pureza = {pureza_media:.6f}")
            if pureza_media > 0.99:
                print(f"  ✅ Estado puro (≈ 1)")
            else:
                print(f"  ⚠️  Estado mixto (pureza < 1)")
        
        # Conservación de energía (todos los tipos)
        if 'energia_total' in self.df.columns:
            E_media = self.df['energia_total'].mean()
            E_std = self.df['energia_total'].std()
            E_var_rel = (E_std / E_media) * 100 if E_media > 0 else 0
            
            print(f"\n✓ Conservación de energía:")
            print(f"  E = {E_media:.6f} ± {E_std:.6f}")
            print(f"  Variación relativa: {E_var_rel:.4f}%")
            if E_var_rel < 1.0:
                print(f"  ✅ CONSERVADA (< 1%)")
            else:
                print(f"  ⚠️  Variación significativa")
        
        print("\n" + "-" * 70)
    
    def analizar_coherente(self):
        """Análisis específico para estado coherente"""
        print("\n📊 Generando gráficas para Estado Coherente...")
        
        fig = plt.figure(figsize=(16, 12))
        
        # 1. Trayectoria en espacio de fase
        ax1 = plt.subplot(3, 3, 1)
        ax1.plot(self.df['X_avg'], self.df['P_avg'], 'r-', alpha=0.7, linewidth=1)
        ax1.plot(self.df['X_avg'].iloc[0], self.df['P_avg'].iloc[0], 'go', markersize=10, label='Inicio')
        ax1.plot(self.df['X_avg'].iloc[-1], self.df['P_avg'].iloc[-1], 'r*', markersize=15, label='Fin')
        ax1.set_xlabel('⟨X⟩')
        ax1.set_ylabel('⟨P⟩')
        ax1.set_title('Trayectoria en Espacio de Fase')
        ax1.legend()
        ax1.grid(True)
        ax1.axis('equal')
        
        # 2. Evolución temporal de cuadraturas
        ax2 = plt.subplot(3, 3, 2)
        ax2.plot(self.df['tiempo'], self.df['X_avg'], 'b-', label='⟨X⟩')
        ax2.plot(self.df['tiempo'], self.df['P_avg'], 'r-', label='⟨P⟩')
        ax2.set_xlabel('Tiempo')
        ax2.set_ylabel('Valor esperado')
        ax2.set_title('Cuadraturas vs Tiempo')
        ax2.legend()
        ax2.grid(True)
        
        # 3. Incertezas
        ax3 = plt.subplot(3, 3, 3)
        ax3.plot(self.df['tiempo'], self.df['delta_X'], 'b-', label='ΔX')
        ax3.plot(self.df['tiempo'], self.df['delta_P'], 'r-', label='ΔP')
        ax3.axhline(y=1/np.sqrt(2), color='g', linestyle='--', label='Teórico')
        ax3.set_xlabel('Tiempo')
        ax3.set_ylabel('Incerteza')
        ax3.set_title('Incertezas')
        ax3.legend()
        ax3.grid(True)
        
        # 4. Producto de incertezas
        ax4 = plt.subplot(3, 3, 4)
        ax4.plot(self.df['tiempo'], self.df['producto_incerteza'], 'purple')
        ax4.axhline(y=0.5, color='r', linestyle='--', label='Límite cuántico')
        ax4.set_xlabel('Tiempo')
        ax4.set_ylabel('ΔX·ΔP')
        ax4.set_title('Principio de Incertidumbre')
        ax4.legend()
        ax4.grid(True)
        
        # 5. Número de fotones
        ax5 = plt.subplot(3, 3, 5)
        ax5.plot(self.df['tiempo'], self.df['n_promedio'], 'orange')
        ax5.set_xlabel('Tiempo')
        ax5.set_ylabel('⟨n⟩')
        ax5.set_title('Número de Fotones')
        ax5.grid(True)
        
        # 6. Mandel Q
        ax6 = plt.subplot(3, 3, 6)
        ax6.plot(self.df['tiempo'], self.df['mandel_Q'], 'brown')
        ax6.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        ax6.set_xlabel('Tiempo')
        ax6.set_ylabel('Mandel Q')
        ax6.set_title('Parámetro de Mandel (Q=0: Poisson)')
        ax6.grid(True)
        
        # 7. Energía
        ax7 = plt.subplot(3, 3, 7)
        ax7.plot(self.df['tiempo'], self.df['energia_total'], 'g-', label='Total')
        ax7.plot(self.df['tiempo'], self.df['energia_cinetica'], 'b--', alpha=0.7, label='Cinética')
        ax7.plot(self.df['tiempo'], self.df['energia_potencial'], 'r--', alpha=0.7, label='Potencial')
        ax7.set_xlabel('Tiempo')
        ax7.set_ylabel('Energía')
        ax7.set_title('Conservación de Energía')
        ax7.legend()
        ax7.grid(True)
        
        # 8. Pureza y Entropía
        ax8 = plt.subplot(3, 3, 8)
        ax81 = ax8
        ax82 = ax8.twinx()
        ax81.plot(self.df['tiempo'], self.df['pureza'], 'b-', label='Pureza')
        ax82.plot(self.df['tiempo'], self.df['entropia'], 'r-', label='Entropía')
        ax81.set_xlabel('Tiempo')
        ax81.set_ylabel('Pureza', color='b')
        ax82.set_ylabel('Entropía', color='r')
        ax81.tick_params(axis='y', labelcolor='b')
        ax82.tick_params(axis='y', labelcolor='r')
        ax8.set_title('Pureza y Entropía')
        ax81.grid(True)
        
        # 9. Parámetro α
        ax9 = plt.subplot(3, 3, 9, projection='polar')
        theta = self.df['alpha_fase'].values
        r = self.df['alpha_magnitud'].values
        scatter = ax9.scatter(theta, r, c=self.df['tiempo'], cmap='viridis', s=10)
        ax9.set_title('α en plano complejo')
        plt.colorbar(scatter, ax=ax9, label='Tiempo')
        
        plt.tight_layout()
        self.fig_principal = fig
    
    def analizar_comprimido(self):
        """Análisis específico para estado comprimido"""
        print("\n📊 Generando gráficas para Estado Comprimido...")
        
        fig = plt.figure(figsize=(16, 12))
        
        ax1 = plt.subplot(3, 3, 1)
        ax1.plot(self.df['X_avg'], self.df['P_avg'], 'r-', alpha=0.7, linewidth=1)
        ax1.plot(self.df['X_avg'].iloc[0], self.df['P_avg'].iloc[0], 'go', markersize=10, label='Inicio')
        ax1.set_xlabel('⟨X⟩')
        ax1.set_ylabel('⟨P⟩')
        ax1.set_title('Trayectoria en Espacio de Fase')
        ax1.legend()
        ax1.grid(True)
        ax1.axis('equal')
        
        ax2 = plt.subplot(3, 3, 2)
        ax2.plot(self.df['tiempo'], self.df['delta_X'], 'b-', label='ΔX')
        ax2.plot(self.df['tiempo'], self.df['delta_P'], 'r-', label='ΔP')
        ax2.set_xlabel('Tiempo')
        ax2.set_ylabel('Incerteza')
        ax2.set_title('Incertezas (Squeezing)')
        ax2.legend()
        ax2.grid(True)
        
        # 3. Squeezing en dB
        ax3 = plt.subplot(3, 3, 3)
        ax3.plot(self.df['tiempo'], self.df['squeezing_dB'], 'purple', linewidth=2)
        ax3.set_xlabel('Tiempo')
        ax3.set_ylabel('Squeezing (dB)')
        ax3.set_title('Squeezing en Decibelios')
        ax3.grid(True)
        
        # 4. Rotación de la elipse
        ax4 = plt.subplot(3, 3, 4)
        ax4.plot(self.df['tiempo'], np.degrees(self.df['theta_ellipse']), 'brown')
        ax4.set_xlabel('Tiempo')
        ax4.set_ylabel('Ángulo (grados)')
        ax4.set_title('Rotación de la Elipse')
        ax4.grid(True)
        
        # 5. Autovalores
        ax5 = plt.subplot(3, 3, 5)
        ax5.plot(self.df['tiempo'], self.df['lambda_1'], 'b-', label='λ₁ (mayor)')
        ax5.plot(self.df['tiempo'], self.df['lambda_2'], 'r-', label='λ₂ (menor)')
        ax5.set_xlabel('Tiempo')
        ax5.set_ylabel('Autovalor')
        ax5.set_title('Autovalores de Σ')
        ax5.legend()
        ax5.grid(True)
        
        # 6. Excentricidad
        ax6 = plt.subplot(3, 3, 6)
        ax6.plot(self.df['tiempo'], self.df['excentricidad'], 'green')
        ax6.set_xlabel('Tiempo')
        ax6.set_ylabel('Excentricidad')
        ax6.set_title('Excentricidad de la Elipse')
        ax6.grid(True)
        
        # 7. Mandel Q (Super-Poisson)
        ax7 = plt.subplot(3, 3, 7)
        ax7.plot(self.df['tiempo'], self.df['mandel_Q'], 'orange')
        ax7.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Poisson')
        ax7.fill_between(self.df['tiempo'], 0, self.df['mandel_Q'].values, 
                         where=self.df['mandel_Q']>0, alpha=0.3, color='red', label='Super-Poisson')
        ax7.set_xlabel('Tiempo')
        ax7.set_ylabel('Mandel Q')
        ax7.set_title('Estadística No-Clásica')
        ax7.legend()
        ax7.grid(True)
        
        # 8. Energía
        ax8 = plt.subplot(3, 3, 8)
        ax8.plot(self.df['tiempo'], self.df['energia_total'], 'g-', linewidth=2)
        ax8.set_xlabel('Tiempo')
        ax8.set_ylabel('Energía Total')
        ax8.set_title('Conservación de Energía')
        ax8.grid(True)
        
        # 9. Área de la elipse
        ax9 = plt.subplot(3, 3, 9)
        ax9.plot(self.df['tiempo'], self.df['area_elipse'], 'purple')
        ax9.axhline(y=np.pi, color='r', linestyle='--', label=f'π = {np.pi:.4f}')
        ax9.set_xlabel('Tiempo')
        ax9.set_ylabel('Área')
        ax9.set_title('Área de la Elipse (Liouville)')
        ax9.legend()
        ax9.grid(True)
        
        plt.tight_layout()
        self.fig_principal = fig
    
    def analizar_superposicion(self):
        """Análisis específico para superposición de estados"""
        print("\n📊 Generando gráficas para Superposición...")
        
        fig = plt.figure(figsize=(16, 14))
        
        # 1. Trayectoria (muestra deformación)
        ax1 = plt.subplot(3, 3, 1)
        scatter = ax1.scatter(self.df['X_avg'], self.df['P_avg'], 
                             c=self.df['tiempo'], cmap='plasma', s=5)
        ax1.set_xlabel('⟨X⟩')
        ax1.set_ylabel('⟨P⟩')
        ax1.set_title('Trayectoria (color = tiempo)')
        plt.colorbar(scatter, ax=ax1, label='Tiempo')
        ax1.grid(True)
        ax1.axis('equal')
        
        # 2. Incertezas (¡se deforman!)
        ax2 = plt.subplot(3, 3, 2)
        ax2.plot(self.df['tiempo'], self.df['delta_X'], 'b-', label='ΔX')
        ax2.plot(self.df['tiempo'], self.df['delta_P'], 'r-', label='ΔP')
        ax2.set_xlabel('Tiempo')
        ax2.set_ylabel('Incerteza')
        ax2.set_title('Incertezas Variables (Deformación)')
        ax2.legend()
        ax2.grid(True)
        
        # 3. Probabilidades de Fock
        ax3 = plt.subplot(3, 3, 3)
        prob_cols = [col for col in self.df.columns if col.startswith('prob_n')]
        for col in prob_cols:
            ax3.plot(self.df['tiempo'], self.df[col], label=col)
        ax3.set_xlabel('Tiempo')
        ax3.set_ylabel('Probabilidad')
        ax3.set_title('Evolución de P(n)')
        ax3.legend()
        ax3.grid(True)
        
        # 4. Coherencias (elementos fuera de diagonal de ρ)
        ax4 = plt.subplot(3, 3, 4)
        coherence_cols = [col for col in self.df.columns if col.startswith('rho_') and col.endswith('_abs')]
        for col in coherence_cols[:5]:  # Primeras 5
            ax4.plot(self.df['tiempo'], self.df[col], label=col.replace('_abs', ''))
        ax4.set_xlabel('Tiempo')
        ax4.set_ylabel('|ρₙₘ|')
        ax4.set_title('Coherencias Cuánticas')
        ax4.legend()
        ax4.grid(True)
        
        # 5. Skewness
        ax5 = plt.subplot(3, 3, 5)
        ax5.plot(self.df['tiempo'], self.df['skewness_X'], 'b-', label='Skew(X)')
        ax5.plot(self.df['tiempo'], self.df['skewness_P'], 'r-', label='Skew(P)')
        ax5.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        ax5.set_xlabel('Tiempo')
        ax5.set_ylabel('Skewness')
        ax5.set_title('Asimetría de Distribución')
        ax5.legend()
        ax5.grid(True)
        
        # 6. Kurtosis
        ax6 = plt.subplot(3, 3, 6)
        ax6.plot(self.df['tiempo'], self.df['kurtosis_X'], 'b-', label='Kurt(X)')
        ax6.plot(self.df['tiempo'], self.df['kurtosis_P'], 'r-', label='Kurt(P)')
        ax6.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Gaussiana')
        ax6.set_xlabel('Tiempo')
        ax6.set_ylabel('Kurtosis')
        ax6.set_title('Grosor de Colas')
        ax6.legend()
        ax6.grid(True)
        
        # 7. Energía
        ax7 = plt.subplot(3, 3, 7)
        ax7.plot(self.df['tiempo'], self.df['energia_total'], 'g-')
        ax7.set_xlabel('Tiempo')
        ax7.set_ylabel('Energía')
        ax7.set_title('Energía Total')
        ax7.grid(True)
        
        # 8. Mandel Q
        ax8 = plt.subplot(3, 3, 8)
        ax8.plot(self.df['tiempo'], self.df['mandel_Q'], 'purple')
        ax8.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        ax8.set_xlabel('Tiempo')
        ax8.set_ylabel('Mandel Q')
        ax8.set_title('Estadística de Fotones')
        ax8.grid(True)
        
        # 9. Matriz de covarianza (evolución de elementos)
        ax9 = plt.subplot(3, 3, 9)
        ax9.plot(self.df['tiempo'], self.df['Sigma_XX'], 'b-', label='Σ_XX')
        ax9.plot(self.df['tiempo'], self.df['Sigma_PP'], 'r-', label='Σ_PP')
        ax9.plot(self.df['tiempo'], self.df['Sigma_XP'], 'g-', label='Σ_XP')
        ax9.set_xlabel('Tiempo')
        ax9.set_ylabel('Elemento de Σ')
        ax9.set_title('Evolución de Matriz de Covarianza')
        ax9.legend()
        ax9.grid(True)
        
        plt.tight_layout()
        self.fig_principal = fig
    
    def analizar_clasico(self):
        """Análisis específico para oscilador clásico"""
        print("\n📊 Generando gráficas para Oscilador Clásico...")
        
        fig = plt.figure(figsize=(16, 10))
        
        # 1. Espacio de fase (x vs v)
        ax1 = plt.subplot(2, 3, 1)
        ax1.plot(self.df['posicion'], self.df['velocidad'], 'b-', linewidth=1)
        ax1.set_xlabel('Posición x')
        ax1.set_ylabel('Velocidad v')
        ax1.set_title('Espacio de Fase (x, v)')
        ax1.grid(True)
        ax1.axis('equal')
        
        # Círculo teórico
        A = self.df['amplitud'].iloc[0]
        omega = self.df['omega'].iloc[0]
        theta = np.linspace(0, 2*np.pi, 100)
        x_circ = A * np.cos(theta)
        v_circ = -A * omega * np.sin(theta)
        ax1.plot(x_circ, v_circ, 'r--', alpha=0.5, label='Teórico')
        ax1.legend()
        
        # 2. Posición vs tiempo
        ax2 = plt.subplot(2, 3, 2)
        ax2.plot(self.df['tiempo'], self.df['posicion'], 'b-')
        ax2.set_xlabel('Tiempo')
        ax2.set_ylabel('Posición x')
        ax2.set_title('Posición vs Tiempo')
        ax2.grid(True)
        
        # 3. Velocidad vs tiempo
        ax3 = plt.subplot(2, 3, 3)
        ax3.plot(self.df['tiempo'], self.df['velocidad'], 'r-')
        ax3.set_xlabel('Tiempo')
        ax3.set_ylabel('Velocidad v')
        ax3.set_title('Velocidad vs Tiempo')
        ax3.grid(True)
        
        # 4. Energía
        ax4 = plt.subplot(2, 3, 4)
        ax4.plot(self.df['tiempo'], self.df['energia_total'], 'g-', label='Total')
        ax4.plot(self.df['tiempo'], self.df['energia_cinetica'], 'b--', alpha=0.7, label='Cinética')
        ax4.plot(self.df['tiempo'], self.df['energia_potencial'], 'r--', alpha=0.7, label='Potencial')
        ax4.set_xlabel('Tiempo')
        ax4.set_ylabel('Energía')
        ax4.set_title('Conservación de Energía')
        ax4.legend()
        ax4.grid(True)
        
        # 5. Fase instantánea
        ax5 = plt.subplot(2, 3, 5)
        ax5.plot(self.df['tiempo'], np.degrees(self.df['fase_instantanea']), 'purple')
        ax5.set_xlabel('Tiempo')
        ax5.set_ylabel('Fase (grados)')
        ax5.set_title('Fase Instantánea')
        ax5.grid(True)
        
        # 6. Distancia al origen
        ax6 = plt.subplot(2, 3, 6)
        ax6.plot(self.df['tiempo'], self.df['distancia_origen'], 'orange')
        ax6.axhline(y=A, color='r', linestyle='--', label='Amplitud')
        ax6.set_xlabel('Tiempo')
        ax6.set_ylabel('r')
        ax6.set_title('Distancia al Origen en Espacio de Fase')
        ax6.legend()
        ax6.grid(True)
        
        plt.tight_layout()
        self.fig_principal = fig
    
    def analisis_fft(self):
        """Análisis de frecuencias usando FFT"""
        print("\n🎵 ANÁLISIS DE FRECUENCIAS (FFT)")
        print("-" * 70)
        
        if self.tipo == 'oscilador_clasico':
            # Para oscilador clásico
            señales = {
                'Posición': self.df['posicion'].values,
                'Velocidad': self.df['velocidad'].values
            }
        else:
            # Para estados cuánticos
            señales = {
                '⟨X⟩': self.df['X_avg'].values,
                '⟨P⟩': self.df['P_avg'].values,
                'ΔX': self.df['delta_X'].values,
                'ΔP': self.df['delta_P'].values
            }
        
        dt = self.df['tiempo'].iloc[1] - self.df['tiempo'].iloc[0]
        N = len(self.df)
        
        fig, axes = plt.subplots(len(señales), 2, figsize=(16, 4*len(señales)))
        if len(señales) == 1:
            axes = axes.reshape(1, -1)
        
        resultados_fft = {}
        
        for idx, (nombre, señal) in enumerate(señales.items()):
            # FFT
            fft_vals = fft(señal)
            fft_freq = fftfreq(N, dt)
            
            # Solo frecuencias positivas
            pos_mask = fft_freq > 0
            fft_freq_pos = fft_freq[pos_mask]
            fft_mag = np.abs(fft_vals[pos_mask])
            fft_power = fft_mag**2
            
            # Encontrar picos
            peaks, properties = find_peaks(fft_power, height=np.max(fft_power)*0.05)
            
            # Guardar resultados
            if len(peaks) > 0:
                freq_dominante = fft_freq_pos[peaks[0]]
                potencia_dominante = fft_power[peaks[0]]
                resultados_fft[nombre] = {
                    'frecuencia': freq_dominante,
                    'potencia': potencia_dominante,
                    'todas_frecuencias': fft_freq_pos[peaks],
                    'todas_potencias': fft_power[peaks]
                }
                
                print(f"\n{nombre}:")
                print(f"  Frecuencia dominante: {freq_dominante:.6f} Hz")
                print(f"  ω = {2*np.pi*freq_dominante:.6f} rad/s")
                print(f"  Período: {1/freq_dominante:.6f} s")
                print(f"  Picos encontrados: {len(peaks)}")
            
            # Gráfica: Señal en el tiempo
            axes[idx, 0].plot(self.df['tiempo'], señal, 'b-', linewidth=1)
            axes[idx, 0].set_xlabel('Tiempo (s)')
            axes[idx, 0].set_ylabel(nombre)
            axes[idx, 0].set_title(f'Señal: {nombre}')
            axes[idx, 0].grid(True)
            
            # Gráfica: Espectro de potencia
            axes[idx, 1].semilogy(fft_freq_pos, fft_power, 'r-', linewidth=1)
            if len(peaks) > 0:
                axes[idx, 1].plot(fft_freq_pos[peaks], fft_power[peaks], 
                                 'go', markersize=10, label='Picos')
                # Anotar frecuencia dominante
                axes[idx, 1].annotate(f'f={fft_freq_pos[peaks[0]]:.4f} Hz',
                                     xy=(fft_freq_pos[peaks[0]], fft_power[peaks[0]]),
                                     xytext=(10, 10), textcoords='offset points',
                                     fontsize=9, color='green',
                                     bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
            axes[idx, 1].set_xlabel('Frecuencia (Hz)')
            axes[idx, 1].set_ylabel('Potencia')
            axes[idx, 1].set_title(f'Espectro de Potencia: {nombre}')
            axes[idx, 1].grid(True)
            axes[idx, 1].legend()
            axes[idx, 1].set_xlim(0, min(5, fft_freq_pos.max()))
        
        plt.tight_layout()
        self.fig_fft = fig
        
        # Análisis con Welch para mejor estimación espectral
        print(f"\n📊 Densidad Espectral de Potencia (Welch):")
        for nombre, señal in list(señales.items())[:2]:  # Solo primeras dos
            f_welch, Pxx = welch(señal, fs=1/dt, nperseg=min(256, N//4))
            peak_idx = np.argmax(Pxx)
            print(f"  {nombre}: f_peak = {f_welch[peak_idx]:.6f} Hz")
        
        print("\n" + "-" * 70)
        return resultados_fft
    
    def ajustar_curvas(self):
        """Ajuste de curvas a funciones teóricas"""
        print("\n📐 AJUSTE DE CURVAS")
        print("-" * 70)
        
        # Función para ajustar oscilaciones
        def oscilacion(t, A, omega, phi, offset):
            return A * np.cos(omega * t + phi) + offset
        
        if self.tipo == 'oscilador_clasico':
            # Ajustar posición
            t = self.df['tiempo'].values
            x = self.df['posicion'].values
            
            # Estimación inicial de parámetros
            A_guess = np.std(x) * np.sqrt(2)
            omega_guess = 2 * np.pi / self.df['periodo'].iloc[0]
            phi_guess = 0
            offset_guess = np.mean(x)
            
            try:
                popt, pcov = curve_fit(oscilacion, t, x, 
                                      p0=[A_guess, omega_guess, phi_guess, offset_guess],
                                      maxfev=5000)
                perr = np.sqrt(np.diag(pcov))
                
                A_fit, omega_fit, phi_fit, offset_fit = popt
                
                print(f"\n🎯 Ajuste de posición x(t) = A·cos(ω·t + φ) + offset:")
                print(f"  A = {A_fit:.6f} ± {perr[0]:.6f}")
                print(f"  ω = {omega_fit:.6f} ± {perr[1]:.6f} rad/s")
                print(f"  φ = {phi_fit:.6f} ± {perr[2]:.6f} rad")
                print(f"  offset = {offset_fit:.6f} ± {perr[3]:.6f}")
                print(f"  Período ajustado: {2*np.pi/omega_fit:.6f} s")
                
                # Calcular R²
                x_fit = oscilacion(t, *popt)
                residuos = x - x_fit
                ss_res = np.sum(residuos**2)
                ss_tot = np.sum((x - np.mean(x))**2)
                r_squared = 1 - (ss_res / ss_tot)
                print(f"  R² = {r_squared:.8f}")
                
                # Gráfica
                fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
                
                ax1.plot(t, x, 'b.', markersize=2, alpha=0.5, label='Datos')
                ax1.plot(t, x_fit, 'r-', linewidth=2, label='Ajuste')
                ax1.set_xlabel('Tiempo (s)')
                ax1.set_ylabel('Posición')
                ax1.set_title(f'Ajuste: x(t) = {A_fit:.3f}·cos({omega_fit:.3f}·t + {phi_fit:.3f}) + {offset_fit:.3f}')
                ax1.legend()
                ax1.grid(True)
                
                ax2.plot(t, residuos, 'g-', linewidth=1)
                ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
                ax2.set_xlabel('Tiempo (s)')
                ax2.set_ylabel('Residuos')
                ax2.set_title(f'Residuos (R² = {r_squared:.6f})')
                ax2.grid(True)
                
                plt.tight_layout()
                self.fig_ajuste = fig
                
            except Exception as e:
                print(f"  ❌ Error en ajuste: {e}")
        
        elif self.tipo in ['estado_coherente', 'estado_comprimido']:
            # Ajustar ⟨X⟩ y ⟨P⟩
            t = self.df['tiempo'].values
            X = self.df['X_avg'].values
            P = self.df['P_avg'].values
            
            # Ajustar X
            A_guess = np.std(X) * np.sqrt(2)
            omega_guess = 1.0  # ω conocido
            
            try:
                popt_X, pcov_X = curve_fit(oscilacion, t, X, 
                                          p0=[A_guess, omega_guess, 0, 0],
                                          maxfev=5000)
                popt_P, pcov_P = curve_fit(oscilacion, t, P, 
                                          p0=[A_guess, omega_guess, -np.pi/2, 0],
                                          maxfev=5000)
                
                print(f"\n🎯 Ajuste de ⟨X⟩:")
                print(f"  A = {popt_X[0]:.6f} ± {np.sqrt(pcov_X[0,0]):.6f}")
                print(f"  ω = {popt_X[1]:.6f} ± {np.sqrt(pcov_X[1,1]):.6f} rad/s")
                
                print(f"\n🎯 Ajuste de ⟨P⟩:")
                print(f"  A = {popt_P[0]:.6f} ± {np.sqrt(pcov_P[0,0]):.6f}")
                print(f"  ω = {popt_P[1]:.6f} ± {np.sqrt(pcov_P[1,1]):.6f} rad/s")
                
                # Gráfica
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
                
                ax1.plot(t, X, 'b.', markersize=2, alpha=0.5, label='Datos ⟨X⟩')
                ax1.plot(t, oscilacion(t, *popt_X), 'r-', linewidth=2, label='Ajuste')
                ax1.set_xlabel('Tiempo (s)')
                ax1.set_ylabel('⟨X⟩')
                ax1.set_title('Ajuste de ⟨X⟩')
                ax1.legend()
                ax1.grid(True)
                
                ax2.plot(t, P, 'b.', markersize=2, alpha=0.5, label='Datos ⟨P⟩')
                ax2.plot(t, oscilacion(t, *popt_P), 'r-', linewidth=2, label='Ajuste')
                ax2.set_xlabel('Tiempo (s)')
                ax2.set_ylabel('⟨P⟩')
                ax2.set_title('Ajuste de ⟨P⟩')
                ax2.legend()
                ax2.grid(True)
                
                plt.tight_layout()
                self.fig_ajuste = fig
                
            except Exception as e:
                print(f"  ❌ Error en ajuste: {e}")
        
        print("\n" + "-" * 70)
    
    def guardar_figuras(self):
        """Guarda las figuras generadas"""
        if not hasattr(self, 'fig_principal'):
            print("⚠️  No hay figuras para guardar")
            return
        
        # Crear carpeta de salida
        output_dir = Path('analisis_output')
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Guardar figura principal
        filename = output_dir / f'{self.nombre}_analisis.png'
        self.fig_principal.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"💾 Figura guardada: {filename}")
    
    def crear_animacion(self, filename='animacion.mp4'):
        """Crea una animación de la evolución del sistema"""
        print(f"\n🎬 Creando animación...")
        
        if self.tipo == 'oscilador_clasico':
            self._animar_clasico(filename)
        else:
            self._animar_cuantico(filename)
    
    def _animar_cuantico(self, filename):
        """Animación para sistemas cuánticos"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # Configurar ejes
        ax1.set_xlabel('⟨X⟩')
        ax1.set_ylabel('⟨P⟩')
        ax1.set_title('Trayectoria en Espacio de Fase')
        ax1.grid(True)
        
        xlim = (self.df['X_avg'].min() - 1, self.df['X_avg'].max() + 1)
        ylim = (self.df['P_avg'].min() - 1, self.df['P_avg'].max() + 1)
        ax1.set_xlim(xlim)
        ax1.set_ylim(ylim)
        ax1.axis('equal')
        
        line, = ax1.plot([], [], 'r-', linewidth=2)
        point, = ax1.plot([], [], 'bo', markersize=10)
        
        # Elipse
        from matplotlib.patches import Ellipse as EllipsePatch
        ellipse = EllipsePatch((0, 0), 0, 0, angle=0, 
                              edgecolor='green', facecolor='none', linewidth=2)
        ax1.add_patch(ellipse)
        
        # Panel derecho: valores
        ax2.axis('off')
        text = ax2.text(0.1, 0.5, '', transform=ax2.transAxes, 
                       fontsize=12, verticalalignment='center', 
                       family='monospace')
        
        def init():
            line.set_data([], [])
            point.set_data([], [])
            return line, point, ellipse, text
        
        def animate(i):
            # Trayectoria acumulada
            x_data = self.df['X_avg'].iloc[:i+1]
            y_data = self.df['P_avg'].iloc[:i+1]
            line.set_data(x_data, y_data)
            
            # Punto actual
            point.set_data([x_data.iloc[-1]], [y_data.iloc[-1]])
            
            # Actualizar elipse
            lambda1 = self.df['lambda_1'].iloc[i]
            lambda2 = self.df['lambda_2'].iloc[i]
            theta = np.degrees(self.df['theta_ellipse'].iloc[i])
            
            ellipse.center = (x_data.iloc[-1], y_data.iloc[-1])
            ellipse.width = 2 * np.sqrt(lambda1)
            ellipse.height = 2 * np.sqrt(lambda2)
            ellipse.angle = theta
            
            # Texto con información
            info = f"""
Tiempo: {self.df['tiempo'].iloc[i]:.3f} s
Frame: {i+1}/{len(self.df)}

⟨X⟩ = {x_data.iloc[-1]:.4f}
⟨P⟩ = {y_data.iloc[-1]:.4f}

ΔX = {self.df['delta_X'].iloc[i]:.4f}
ΔP = {self.df['delta_P'].iloc[i]:.4f}
ΔX·ΔP = {self.df['producto_incerteza'].iloc[i]:.4f}

⟨n⟩ = {self.df['n_promedio'].iloc[i]:.4f}
E = {self.df['energia_total'].iloc[i]:.4f}
            """
            text.set_text(info)
            
            return line, point, ellipse, text
        
        # Reducir frames para animación más rápida
        skip = max(1, len(self.df) // 300)  # ~300 frames máximo
        
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                      frames=range(0, len(self.df), skip),
                                      interval=20, blit=True)
        
        anim.save(filename, writer='ffmpeg', fps=30, dpi=150)
        print(f"✅ Animación guardada: {filename}")
        plt.close(fig)
    
    def _animar_clasico(self, filename):
        """Animación para oscilador clásico"""
        print("Animación clásica no implementada aún")
    
    def exportar_resumen(self, filename='resumen_analisis.txt'):
        """Exporta un resumen del análisis a archivo de texto"""
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"RESUMEN DE ANÁLISIS - {self.nombre}\n")
            f.write("=" * 70 + "\n\n")
            f.write(f"Tipo de simulación: {self.tipo}\n")
            f.write(f"Puntos de datos: {len(self.df)}\n")
            f.write(f"Tiempo total: {self.df['tiempo'].max():.4f} s\n")
            f.write(f"Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            # Estadísticas principales
            if self.tipo != 'oscilador_clasico':
                f.write("ESTADÍSTICAS CUÁNTICAS\n")
                f.write("-" * 70 + "\n")
                f.write(f"⟨X⟩ promedio: {self.df['X_avg'].mean():.6f}\n")
                f.write(f"⟨P⟩ promedio: {self.df['P_avg'].mean():.6f}\n")
                f.write(f"ΔX promedio: {self.df['delta_X'].mean():.6f}\n")
                f.write(f"ΔP promedio: {self.df['delta_P'].mean():.6f}\n")
                f.write(f"ΔX·ΔP promedio: {self.df['producto_incerteza'].mean():.6f}\n")
                f.write(f"Energía promedio: {self.df['energia_total'].mean():.6f}\n")
                f.write(f"Pureza promedio: {self.df['pureza'].mean():.6f}\n")
        
        print(f"📄 Resumen exportado: {filename}")

In [13]:
class ComparadorSimulaciones:
    """Clase para comparar múltiples simulaciones"""
    
    def __init__(self, archivos):
        """
        Inicializa el comparador con múltiples archivos
        
        Args:
            archivos (list): Lista de rutas a archivos CSV
        """
        self.archivos = archivos
        self.analizadores = []
        
        print(f"\n{'='*70}")
        print(f"Comparando {len(archivos)} simulaciones")
        print(f"{'='*70}\n")
        
        for archivo in archivos:
            try:
                analizador = AnalizadorQHO(archivo)
                self.analizadores.append(analizador)
            except Exception as e:
                print(f"❌ Error cargando {archivo}: {e}")
    
    def comparar_trayectorias(self):
        """Compara trayectorias en espacio de fase"""
        print("\n🔄 COMPARACIÓN DE TRAYECTORIAS")
        print("-" * 70)
        
        fig, ax = plt.subplots(figsize=(12, 10))
        
        colores = plt.cm.tab10(np.linspace(0, 1, len(self.analizadores)))
        
        for idx, (analizador, color) in enumerate(zip(self.analizadores, colores)):
            df = analizador.df
            
            if analizador.tipo == 'oscilador_clasico':
                x = df['posicion'].values
                p = df['velocidad'].values
                label = f'{analizador.nombre} (Clásico)'
            else:
                x = df['X_avg'].values
                p = df['P_avg'].values
                label = f'{analizador.nombre} ({analizador.tipo})'
            
            # Plotear trayectoria
            ax.plot(x, p, '-', color=color, linewidth=2, alpha=0.7, label=label)
            ax.plot(x[0], p[0], 'o', color=color, markersize=10)
            ax.plot(x[-1], p[-1], 's', color=color, markersize=10)
        
        ax.set_xlabel('X / x')
        ax.set_ylabel('P / v')
        ax.set_title('Comparación de Trayectorias en Espacio de Fase')
        ax.legend(loc='best')
        ax.grid(True)
        ax.axis('equal')
        
        plt.tight_layout()
        self.fig_trayectorias = fig
        print("✅ Gráfica de trayectorias generada")
    
    def comparar_incertezas(self):
        """Compara evolución de incertezas"""
        print("\n📊 COMPARACIÓN DE INCERTEZAS")
        print("-" * 70)
        
        # Filtrar solo simulaciones cuánticas
        cuanticos = [a for a in self.analizadores if a.tipo != 'oscilador_clasico']
        
        if len(cuanticos) == 0:
            print("⚠️  No hay simulaciones cuánticas para comparar incertezas")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        colores = plt.cm.tab10(np.linspace(0, 1, len(cuanticos)))
        
        for analizador, color in zip(cuanticos, colores):
            df = analizador.df
            t = df['tiempo'].values
            
            # ΔX
            axes[0, 0].plot(t, df['delta_X'], '-', color=color, 
                           linewidth=2, label=analizador.nombre)
            
            # ΔP
            axes[0, 1].plot(t, df['delta_P'], '-', color=color, 
                           linewidth=2, label=analizador.nombre)
            
            # ΔX·ΔP
            axes[1, 0].plot(t, df['producto_incerteza'], '-', color=color, 
                           linewidth=2, label=analizador.nombre)
            
            # Pureza
            axes[1, 1].plot(t, df['pureza'], '-', color=color, 
                           linewidth=2, label=analizador.nombre)
        
        # Configurar ejes
        axes[0, 0].set_xlabel('Tiempo (s)')
        axes[0, 0].set_ylabel('ΔX')
        axes[0, 0].set_title('Incerteza en X')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        axes[0, 0].axhline(y=1/np.sqrt(2), color='k', linestyle='--', alpha=0.3)
        
        axes[0, 1].set_xlabel('Tiempo (s)')
        axes[0, 1].set_ylabel('ΔP')
        axes[0, 1].set_title('Incerteza en P')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
        axes[0, 1].axhline(y=1/np.sqrt(2), color='k', linestyle='--', alpha=0.3)
        
        axes[1, 0].set_xlabel('Tiempo (s)')
        axes[1, 0].set_ylabel('ΔX·ΔP')
        axes[1, 0].set_title('Producto de Incertezas')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
        axes[1, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='Límite cuántico')
        
        axes[1, 1].set_xlabel('Tiempo (s)')
        axes[1, 1].set_ylabel('Pureza')
        axes[1, 1].set_title('Pureza del Estado')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        self.fig_incertezas = fig
        print("✅ Gráfica de incertezas generada")
    
    def comparar_energias(self):
        """Compara conservación de energía"""
        print("\n⚡ COMPARACIÓN DE ENERGÍAS")
        print("-" * 70)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        colores = plt.cm.tab10(np.linspace(0, 1, len(self.analizadores)))
        
        for analizador, color in zip(self.analizadores, colores):
            df = analizador.df
            t = df['tiempo'].values
            E = df['energia_total'].values
            
            # Energía vs tiempo
            ax1.plot(t, E, '-', color=color, linewidth=2, label=analizador.nombre)
            
            # Variación relativa
            E_mean = np.mean(E)
            E_rel = (E - E_mean) / E_mean * 100
            ax2.plot(t, E_rel, '-', color=color, linewidth=2, label=analizador.nombre)
            
            # Estadísticas
            print(f"\n{analizador.nombre}:")
            print(f"  E promedio: {E_mean:.6f}")
            print(f"  Desv. estándar: {np.std(E):.6f}")
            print(f"  Variación rel: {np.std(E)/E_mean*100:.4f}%")
        
        ax1.set_xlabel('Tiempo (s)')
        ax1.set_ylabel('Energía Total')
        ax1.set_title('Conservación de Energía')
        ax1.legend()
        ax1.grid(True)
        
        ax2.set_xlabel('Tiempo (s)')
        ax2.set_ylabel('Variación Relativa (%)')
        ax2.set_title('Variación Relativa de Energía')
        ax2.legend()
        ax2.grid(True)
        ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
        
        plt.tight_layout()
        self.fig_energias = fig
        print("\n✅ Gráfica de energías generada")
        print("-" * 70)
    
    def comparar_estadistica_fotones(self):
        """Compara estadística de fotones (solo estados cuánticos)"""
        print("\n🔢 COMPARACIÓN DE ESTADÍSTICA DE FOTONES")
        print("-" * 70)
        
        cuanticos = [a for a in self.analizadores if a.tipo != 'oscilador_clasico']
        
        if len(cuanticos) == 0:
            print("⚠️  No hay simulaciones cuánticas")
            return
        
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
        
        colores = plt.cm.tab10(np.linspace(0, 1, len(cuanticos)))
        
        for analizador, color in zip(cuanticos, colores):
            df = analizador.df
            t = df['tiempo'].values
            
            # ⟨n⟩
            axes[0].plot(t, df['n_promedio'], '-', color=color, 
                        linewidth=2, label=analizador.nombre)
            
            # Mandel Q
            axes[1].plot(t, df['mandel_Q'], '-', color=color, 
                        linewidth=2, label=analizador.nombre)
            
            # Fano F
            axes[2].plot(t, df['fano_F'], '-', color=color, 
                        linewidth=2, label=analizador.nombre)
        
        axes[0].set_xlabel('Tiempo (s)')
        axes[0].set_ylabel('⟨n⟩')
        axes[0].set_title('Número Promedio de Fotones')
        axes[0].legend()
        axes[0].grid(True)
        
        axes[1].set_xlabel('Tiempo (s)')
        axes[1].set_ylabel('Mandel Q')
        axes[1].set_title('Parámetro de Mandel')
        axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Poisson')
        axes[1].legend()
        axes[1].grid(True)
        
        axes[2].set_xlabel('Tiempo (s)')
        axes[2].set_ylabel('Fano F')
        axes[2].set_title('Factor de Fano')
        axes[2].axhline(y=1, color='k', linestyle='--', alpha=0.3, label='Poisson')
        axes[2].legend()
        axes[2].grid(True)
        
        plt.tight_layout()
        self.fig_fotones = fig
        print("✅ Gráfica de estadística de fotones generada")
        print("-" * 70)
    
    def tabla_comparativa(self):
        """Genera tabla comparativa con estadísticas"""
        print("\n📋 TABLA COMPARATIVA")
        print("=" * 100)
        
        # Crear DataFrame para comparación
        datos = []
        
        for analizador in self.analizadores:
            df = analizador.df
            
            fila = {
                'Simulación': analizador.nombre,
                'Tipo': analizador.tipo,
                'Puntos': len(df),
                'Tiempo total': f"{df['tiempo'].max():.2f} s",
                'E promedio': f"{df['energia_total'].mean():.6f}",
                'E desv.std': f"{df['energia_total'].std():.6f}",
            }
            
            if analizador.tipo != 'oscilador_clasico':
                fila.update({
                    '⟨n⟩ promedio': f"{df['n_promedio'].mean():.4f}",
                    'ΔX·ΔP min': f"{df['producto_incerteza'].min():.6f}",
                    'Pureza': f"{df['pureza'].mean():.6f}",
                    'Entropía': f"{df['entropia'].mean():.6f}"
                })
            
            datos.append(fila)
        
        tabla = pd.DataFrame(datos)
        print(tabla.to_string(index=False))
        print("=" * 100)
        
        # Guardar tabla
        tabla.to_csv('comparacion_simulaciones.csv', index=False)
        print("\n💾 Tabla guardada: comparacion_simulaciones.csv")
    
    def guardar_figuras_comparativas(self):
        """Guarda todas las figuras comparativas"""
        output_dir = Path('Proyecto/Figs/comparaciones_output')
        output_dir.mkdir(parents=True, exist_ok=True)
        
        figuras = ['fig_trayectorias', 'fig_incertezas', 'fig_energias', 'fig_fotones']
        
        for fig_name in figuras:
            if hasattr(self, fig_name):
                fig = getattr(self, fig_name)
                filename = output_dir / f'{fig_name}.png'
                fig.savefig(filename, dpi=300, bbox_inches='tight')
                print(f"💾 Figura guardada: {filename}")

In [14]:
def comparar_multiples(archivos):
    """Función para comparar múltiples simulaciones"""
    comparador = ComparadorSimulaciones(archivos)
    
    comparador.comparar_trayectorias()
    comparador.comparar_incertezas()
    comparador.comparar_energias()
    comparador.comparar_estadistica_fotones()
    comparador.tabla_comparativa()
    comparador.guardar_figuras_comparativas()
    
    print("\n✅ Comparación completa finalizada!")
    return comparador

In [19]:
def analizar_directorio(directorio='CSV'):
    """Analiza todos los archivos CSV en un directorio"""
    archivos_csv = list(Path(directorio).glob('*.csv'))
    
    if not archivos_csv:
        print(f"❌ No se encontraron archivos CSV en {directorio}")
        return
    
    print(f"\n🔍 Encontrados {len(archivos_csv)} archivos CSV\n")
    
    # Crear carpeta de salida
    output_dir = Path('analisis_output')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    for idx, archivo in enumerate(archivos_csv, 1):
        print(f"\n{'='*70}")
        print(f"Analizando archivo {idx}/{len(archivos_csv)}: {archivo.name}")
        print(f"{'='*70}")
        
        try:
            analizador = AnalizadorQHO(str(archivo))
            
            # 1. Análisis completo
            analizador.analisis_completo(guardar=True)
            
            # 2. FFT
            print("\n🎵 Ejecutando análisis FFT...")
            analizador.analisis_fft()
            
            # 3. Ajuste de curvas
            print("\n📐 Ejecutando ajuste de curvas...")
            analizador.ajustar_curvas()
            
            # 4. Exportar resumen
            resumen_file = output_dir / f'resumen_{archivo.stem}.txt'
            analizador.exportar_resumen(str(resumen_file))
            
            # 5. Guardar figuras FFT y Ajustes
            if hasattr(analizador, 'fig_fft'):
                fft_file = output_dir / f'fft_{archivo.stem}.png'
                analizador.fig_fft.savefig(fft_file, dpi=300, bbox_inches='tight')
                print(f"💾 FFT guardado: {fft_file}")
            
            if hasattr(analizador, 'fig_ajuste'):
                ajuste_file = output_dir / f'ajuste_{archivo.stem}.png'
                analizador.fig_ajuste.savefig(ajuste_file, dpi=300, bbox_inches='tight')
                print(f"💾 Ajuste guardado: {ajuste_file}")
            
            print(f"\n✅ Análisis de {archivo.name} completado")
            
        except Exception as e:
            print(f"❌ Error al analizar {archivo.name}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    print("\n" + "="*70)
    print("✅ Análisis completo de todos los archivos finalizado!")
    print("="*70)

In [20]:
if __name__ == '__main__':
    print("""
    ╔═══════════════════════════════════════════════════════════════╗
    ║  Análisis Automático de Simulaciones QHO                     ║
    ║  Oscilador Armónico Cuántico - Estados Coherentes           ║
    ╚═══════════════════════════════════════════════════════════════╝
    """)
    
    # Crear carpetas de salida
    Path('analisis_output').mkdir(parents=True, exist_ok=True)
    Path('Figs/comparaciones_output').mkdir(parents=True, exist_ok=True)
    
    if len(sys.argv) > 1:
        # ===== MODO 1: COMPARACIÓN MÚLTIPLE =====
        if sys.argv[1] == '--comparar':
            if len(sys.argv) < 4:
                print("❌ Uso: python analisis_qho.py --comparar archivo1.csv archivo2.csv ...")
                sys.exit(1)
            
            archivos = sys.argv[2:]
            print(f"\n🔄 Modo Comparación: {len(archivos)} archivos\n")
            comparador = comparar_multiples(archivos)
            plt.show()
            sys.exit(0)  # ✅ Terminar aquí
        
        # ===== MODO 2: ANÁLISIS INDIVIDUAL =====
        else:
            archivo = sys.argv[1]
            
            if not os.path.exists(archivo):
                print(f"❌ Archivo no encontrado: {archivo}")
                sys.exit(1)
            
            print(f"\n📊 Modo Análisis Individual: {archivo}\n")
            
            # Crear analizador
            analizador = AnalizadorQHO(archivo)
            
            # 1. Análisis completo (gráficas principales)
            analizador.analisis_completo(guardar=True)
            
            # 2. FFT
            print("\n🎵 Ejecutando análisis FFT...")
            analizador.analisis_fft()
            
            # 3. Ajuste de curvas
            print("\n📐 Ejecutando ajuste de curvas...")
            analizador.ajustar_curvas()
            
            # 4. Exportar resumen
            resumen_file = f'analisis_output/resumen_{analizador.nombre}.txt'
            analizador.exportar_resumen(resumen_file)
            
            # 5. Guardar figuras adicionales (FFT y Ajustes)
            output_dir = Path('analisis_output')
            
            if hasattr(analizador, 'fig_fft'):
                fft_file = output_dir / f'fft_{analizador.nombre}.png'
                analizador.fig_fft.savefig(fft_file, dpi=300, bbox_inches='tight')
                print(f"💾 FFT guardado: {fft_file}")
            
            if hasattr(analizador, 'fig_ajuste'):
                ajuste_file = output_dir / f'ajuste_{analizador.nombre}.png'
                analizador.fig_ajuste.savefig(ajuste_file, dpi=300, bbox_inches='tight')
                print(f"💾 Ajuste guardado: {ajuste_file}")
            
            # 6. Preguntar por animación
            print("\n" + "="*70)
            respuesta = input("¿Crear animación? (s/n): ").lower()
            if respuesta == 's':
                anim_file = output_dir / f'animacion_{analizador.nombre}.mp4'
                analizador.crear_animacion(str(anim_file))
            
            print("\n✅ Análisis individual completo!")
            plt.show()
    
    else:
        # ===== MODO 3: ANALIZAR TODO EL DIRECTORIO =====
        print("\n📁 Modo Directorio: Analizando todos los CSV\n")
        analizar_directorio()
        plt.show()


    ╔═══════════════════════════════════════════════════════════════╗
    ║  Análisis Automático de Simulaciones QHO                     ║
    ║  Oscilador Armónico Cuántico - Estados Coherentes           ║
    ╚═══════════════════════════════════════════════════════════════╝
    
❌ Archivo no encontrado: --f=c:\Users\mario\AppData\Roaming\jupyter\runtime\kernel-v3943b0b3cd11625579fea7e6b81d7fcdad65d014b.json


SystemExit: 1